


## **Practice 00 Training MLP without Keras**

practice with small data set

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/miokobayashii/summerschool2022/blob/main/day01/Practice02_DefineLossFunctionOptimizer.ipynb)


<img src="https://docs.google.com/uc?id=1F55coG7rc2RWMBSVgoUy_3haGRUP63R8" width="500">


## **Practice 00 Training process with iris dataset**

###As a training data set, we use Iris data. 

   1. sepal length in cm
   2. sepal width in cm
   3. petal length in cm
   4. petal width in cm
   5. class: \
      -- Iris Setosa \
      -- Iris Versicolour \
      -- Iris Virginica

We will use "Petal length" and "Petal width" for input data, \
and we define that if the input values represents \
iris "Setosa", output y = 1, otherwise, 0. 

<img src="https://upload.wikimedia.org/wikipedia/commons/5/56/Iris_dataset_scatterplot.svg" width="500">

Setosa 
<img src="https://upload.wikimedia.org/wikipedia/commons/5/56/Kosaciec_szczecinkowaty_Iris_setosa.jpg" width="100"> 
Versicolor 
<img src="https://upload.wikimedia.org/wikipedia/commons/4/41/Iris_versicolor_3.jpg" width="100">
Virginica 
<img src="https://upload.wikimedia.org/wikipedia/commons/9/9f/Iris_virginica.jpg" width="100">

In [249]:
#from keras.layers import Dense
#from keras.layers.advanced_activations import LeakyReLU
#from keras.models import Sequential

#from keras.optimizers import adam_v2

#from tensorflow.keras import initializers
#from keras.utils.vis_utils import plot_model
import numpy as np
import matplotlib.pyplot as plt

from sklearn import datasets
from sklearn.model_selection import train_test_split
from keras.utils import np_utils

In [267]:
iris = datasets.load_iris() # load data
x = iris.data   # features of iris
y = iris.target # label 0, 1, 2

# Label: Setosa = 0, Versicolor or Virginica = 1
y = np.where(y == 2, 1, y)
x = np.array(x[:,2:])
x_train, x_test, y_train, y_test= train_test_split(x, y, train_size=0.8)  

In [242]:
x.shape, y.shape,

((150, 2), (150,))

In [270]:
alpha = 0.01
lr = 0.001


In [225]:
def LeakyReLU(x,alpha):
  return np.where(x < 0, alpha*x, x)

In [99]:
def sigmoid(x):
  return 1/(1 + np.exp(-x)) 

In [268]:
#np.random.seed(seed=32)
param_w = np.random.randn(4,2)
param_b1 = np.random.randn(4)
param_v = np.random.randn(4,1)
param_b2 = np.random.randn(1)

In [229]:
param_w,param_b1

(array([[-0.34889445,  0.98370343],
        [ 0.58092283,  0.07028444],
        [ 0.77753268,  0.58195875],
        [ 1.47179053,  1.66318101]]),
 array([-0.26117712, -0.68867681, -0.69492326,  1.94042346]))

In [230]:
param_v, param_b2

(array([[ 1.80541519],
        [ 0.45631385],
        [-0.57481204],
        [ 0.1141805 ]]), array([1.51358077]))

### check

In [140]:
param_w.shape,param_b1.shape,param_v.shape, param_b2.shape

((4, 2), (4,), (4, 1), (1,))

In [141]:
x_train[0]

array([4.5, 1.5])

In [142]:
(param_w@x_train[0] + param_b1).shape

(4,)

In [271]:
def feedForward(x, param_w, param_b1,param_v, param_b2):
  s0 = param_w@x + param_b1
  m0 = LeakyReLU(s0, alpha)
  y_h = sigmoid(m0 @ param_v + param_b2)
  return y_h, m0, s0

In [272]:
def dEdv(y_h, y, m0):
  return (y_h - y)*m0, (y_h - y) ## gradients with respect to weights and biases


In [145]:
def dEdw(x, y, y_h, s0, param_v):  
  vi = np.where(s0.reshape(4,1) > 0, param_v, param_v*alpha).reshape(4,1)
  return (vi*(y_h-y))*x, (vi*(y_h-y))

In [146]:
def Loss(y, y_h):
  return -y*np.log(y_h) - (1-y)*np.log(1-y_h)


In [ ]:
np.arange(x_train.shape[0])

In [273]:
def pLoss(x_train,y_train,param_w, param_b1,param_v, param_b2, count, epoch):
  dloss = []
  for i in np.arange(x_train.shape[0]):
    y_h, _, _ = feedForward(x_train[i],param_w, param_b1,param_v, param_b2)
    dloss.append(Loss(y_train[i], y_h))
  
  if count % (x_train.shape[0]) == 0:
    print(epoch, np.array(dloss).sum()/x_train.shape[0], end=" ")
    acc(x_test,param_w, param_b1,param_v, param_b2)
  return np.array(dloss).sum()/x_train.shape[0]

In [274]:
pLoss(x_train,y_train,param_w, param_b1,param_v, param_b2, 0, 0)

0 1.9686412767312846 16.0
Accuracy: 53.333333333333336


1.9686412767312846

In [275]:
res = 10
epochs = 20
for j in range(epochs):
  
  for i in np.arange(x_train.shape[0]):
    
    res = pLoss(x_train,y_train,param_w, param_b1,param_v, param_b2, i, j)
    if(res < 0.2): break
    
    y_h, m0, s0 = feedForward(x_train[i],param_w, param_b1,param_v, param_b2)

    gd_v, gd_b2 = dEdv(y_h, y[i], m0)
    gd_v= gd_v.reshape(4,1)
    gd_b2 = gd_b2.reshape(1,)

    gd_w, gd_b1 = dEdw(x_train[i], y[i], y_h, s0, param_v)
    gd_b1 = gd_b1.reshape(4,)

    param_w = param_w - lr * gd_w
    param_b1= param_b1 - lr * gd_b1
    param_v = param_v - lr * gd_v
    param_b2= param_b2 - lr * gd_b2
    #print(param_w.shape,param_b1.shape,param_v.shape, param_b2.shape)


0 1.9686412767312846 16.0
Accuracy: 53.333333333333336
1 1.5558269525209523 16.0
Accuracy: 53.333333333333336
2 1.2487971936754187 16.0
Accuracy: 53.333333333333336
3 0.9876035204421032 16.0
Accuracy: 53.333333333333336
4 0.7721742734740759 16.0
Accuracy: 53.333333333333336
5 0.6206696405066595 20.0
Accuracy: 66.66666666666667
6 0.5304256985647549 29.0
Accuracy: 96.66666666666667
7 0.48142800512174894 30.0
Accuracy: 100.0
8 0.45549895915308086 30.0
Accuracy: 100.0
9 0.44173897340954144 30.0
Accuracy: 100.0
10 0.4344080318497417 30.0
Accuracy: 100.0
11 0.43053892871090205 30.0
Accuracy: 100.0
12 0.4285693826763094 30.0
Accuracy: 100.0
13 0.42765671824881113 30.0
Accuracy: 100.0
14 0.42733735824062646 30.0
Accuracy: 100.0
15 0.42735274278839686 30.0
Accuracy: 100.0
16 0.4275573530984659 30.0
Accuracy: 100.0
17 0.4278688116726837 30.0
Accuracy: 100.0
18 0.4282403057172862 30.0
Accuracy: 100.0
19 0.4286451623821207 30.0
Accuracy: 100.0


In [277]:
def acc(x_test,param_w, param_b1,param_v, param_b2):
  acc = 0
  #print(x_test.shape[0])
  for i in np.arange(x_test.shape[0]):
    [yout], _, _ = feedForward(x_test[i],param_w, param_b1,param_v, param_b2)
    acc += 1- np.abs(np.floor(yout+0.5) - y_test[i])

  print(acc)
  print("Accuracy:", 100*acc/x_test.shape[0])

  
acc(x_test,param_w, param_b1,param_v, param_b2)

30.0
Accuracy: 100.0


In [172]:
print(param_w,param_b1)
print(param_v, param_b2)

[[ 0.64163957  0.30826504]
 [ 0.46208351  0.92285173]
 [ 0.56965527  0.76555373]
 [-0.25304071  0.83965929]] [0.55089823 0.57345406 0.047399   0.23889998]
[[ 0.05175754]
 [-0.11265294]
 [ 0.02109115]
 [ 0.57401076]] [0.4089091]




---



### We can train the model with "model.fit()"

A batch means a set of training data for one forward pass and one backword pass. If batch size is 8, we use 8 training data to traing the model.

An epoch means training the model with all training data for one cycle.



In [ ]:
plt.plot(history.epoch, history.history["accuracy"], label="acc")
plt.plot(history.epoch, history.history["loss"], label="loss")
plt.xlabel("epoch")
plt.legend()

In [ ]:
# Evaluation with test data
score = model.evaluate(x_test, y_test, verbose=1)
print("Test loss", score[0])
print("Test accuracy", score[1])

# Test inidividual data
print("====================================")
print("-----------correct answer-----------")
print(y_test[0])
print(y_test[7])
print("-----------predict answer-----------")
print(np.round(model.predict(x_test)[0]))
print(np.round(model.predict(x_test)[7]))

About accuracy: https://keras.io/api/metrics/accuracy_metrics/

In [ ]:
model = Sequential()

model.add(Dense(4, input_shape=(2,)))

# Leaky ReLU activation
model.add(LeakyReLU(alpha=0.01))

model.add(Dense(1, activation='sigmoid'))


In [ ]:
model.compile(
    optimizer=adam_v2.Adam(learning_rate=0.0001), 
    loss='               ', 
    metrics=['       ']
)

In [ ]:
model.summary()
plot_model(model, to_file='model.png', show_shapes=True)

In [ ]:
history = model.fit(x_train, y_train, epochs=500, batch_size=8)

## Please write down your consideration and comments here:




